# LBT Demo: Block Sizes 16, 8, 4
Impacto del tamaño de bloque (Block Size `-B`) en el Learned Block Transform (LBT).

Ejecución del codificador LBT con tres tamaños de bloque diferentes y muestra los resultados visuales y el tamaño de los archivos generados.

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import sys
sys.path.append('../src')
from visualization import show_images

In [ ]:
# Configuración de rutas
src_dir = '../src'
docs_dir = '../docs'
img_file = 'hor.jpg'
input_img = f'{docs_dir}/hor.jpg'

# Directorios de salida (rutas relativas al directorio src)
encoded_dir_rel = f'{docs_dir}/encoded'
decoded_dir_rel = f'{docs_dir}/decoded'

# Crear directorios si no existen
# Nota: Estamos en notebooks/, así que ../docs es la ruta relativa correcta
os.makedirs(f'../docs/encoded', exist_ok=True)
os.makedirs(f'../docs/decoded', exist_ok=True)

## 1. Bloque 16x16

In [ ]:
# Tamaño de bloque 16x16
B = 16
qs = 16  # Quantization Step Size (controlar la calidad/compresión.)
enc_file = f'{encoded_dir_rel}/demo_B{B}'
dec_file = f'{decoded_dir_rel}/demo_B{B}.jpg'

# Ejecutar codificador y decodificador LBT
!cd {src_dir} && python LBT.py encode -o {input_img} -e {enc_file} -B {B} -q {qs}
!cd {src_dir} && python LBT.py decode -e {enc_file} -d {dec_file} -B {B} -q {qs}

# Mostrar resultados visuales (las rutas son relativas al notebook)
show_images(f'../docs/{img_file}', f'../docs/encoded/demo_B{B}.tif', f'../docs/decoded/demo_B{B}.jpg', f'Tamaño de Bloque {B} (QSS={qs})')

## 2. Bloque 8x8 (Estándar)

In [ ]:
# Tamaño de bloque 8x8 (estándar en compresión de imágenes, como en JPEG)
B = 8
enc_file = f'{encoded_dir_rel}/demo_B{B}'
dec_file = f'{decoded_dir_rel}/demo_B{B}.jpg'

# Ejecutar codificador y decodificador LBT
!cd {src_dir} && python LBT.py encode -o {input_img} -e {enc_file} -B {B} -q {qs}
!cd {src_dir} && python LBT.py decode -e {enc_file} -d {dec_file} -B {B} -q {qs}

# Mostrar y comparar resultados
show_images(f'../docs/{img_file}', f'../docs/encoded/demo_B{B}.tif', f'../docs/decoded/demo_B{B}.jpg', f'Tamaño de Bloque {B} (QSS={qs})')

## 3. Bloque 4x4

In [ ]:
# Tamaño de bloque 4x4 (bloques más pequeños, mayor complejidad computacional)
B = 4
enc_file = f'{encoded_dir_rel}/demo_B{B}'
dec_file = f'{decoded_dir_rel}/demo_B{B}.jpg'

# Ejecutar codificador y decodificador LBT
!cd {src_dir} && python LBT.py encode -o {input_img} -e {enc_file} -B {B} -q {qs}
!cd {src_dir} && python LBT.py decode -e {enc_file} -d {dec_file} -B {B} -q {qs}

# Mostrar y comparar resultados
show_images(f'../docs/{img_file}', f'../docs/encoded/demo_B{B}.tif', f'../docs/decoded/demo_B{B}.jpg', f'Tamaño de Bloque {B} (QSS={qs})')